In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import pandas as pd
import time

In [13]:
driver = webdriver.Chrome()
url = 'https://www.op.gg/leaderboards/tier?hl=ko_KR&tier=challenger&page=1'
driver.get(url)

In [17]:
# 확인
user_name = driver.find_element(By.CLASS_NAME, 'whitespace-pre-wrap').text
tier = driver.find_element(By.CLASS_NAME, 'hidden text-gray-600').text
lp = driver.find_element(By.CLASS_NAME, 'text-gray-600').text

row = driver.find_element(By.CSS_SELECTOR, 'table tbody tr')
imgs = row.find_elements(By.CSS_SELECTOR, 'flex items-center')
champs = [img.get_attribute('alt') for img in imgs]
most = ', '.join(champs)

winning_rate = driver.find_element(By.CLASS_NAME, 'css-1ex2x1n').text
win_parts = winning_rate.splitlines() # 개행 기준
win = win_parts[0].replace('W', '')
lose = win_parts[1].replace('L', '')
rate = win_parts[2].replace('%', '')

print(f"유저 이름: {user_name}\n티어: {tier}\nLP: {lp}\n모스트 챔피언: {most}\n승리 수: {win}\n패배 수: {lose}\n승률: {rate}")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".hidden text-gray-600"}
  (Session info: chrome=134.0.6998.166); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010327f6c8 cxxbridge1$str$ptr + 2791212
1   chromedriver                        0x0000000103277c9c cxxbridge1$str$ptr + 2759936
2   chromedriver                        0x0000000102dc9e30 cxxbridge1$string$len + 92928
3   chromedriver                        0x0000000102e11170 cxxbridge1$string$len + 384576
4   chromedriver                        0x0000000102e525f8 cxxbridge1$string$len + 651976
5   chromedriver                        0x0000000102e052fc cxxbridge1$string$len + 335820
6   chromedriver                        0x00000001032446c4 cxxbridge1$str$ptr + 2549544
7   chromedriver                        0x

In [12]:
# 브라우저 실행
options = Options()
driver = webdriver.Chrome(options=options)

# 기본 설정
base_url = 'https://www.op.gg/leaderboards/tier?page={}'
data_list = []
page = 1
stop_tier_full = "emerald 4"

try:
    while True:
        print(f"\n[페이지 {page}] 접속 중...")
        driver.get(base_url.format(page))

        try:
            # 테이블 로딩될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody > tr'))
            )
        except:
            print(f"[INFO] 페이지 {page} 로딩 실패 또는 데이터 없음 ❌")
            break

        rows = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
        if len(rows) == 0:
            print(f"[INFO] 더 이상 데이터가 없습니다. (페이지 {page}) ❌")
            break

        print(f"[INFO] {len(rows)}개의 랭커 항목 수집 중")

        for idx, row in enumerate(rows, 1):
            try:
                cells = row.find_elements(By.TAG_NAME, 'td')
                if len(cells) < 7:
                    continue

                rank = cells[0].text.strip()
                summoner_tag = cells[1].text.strip()
                tier = cells[2].text.strip()
                lp = cells[3].text.strip()
                level = cells[5].text.strip()
                win_info = cells[6].text.strip()

                # 챔피언 이미지 alt 속성 수집
                champ_imgs = cells[4].find_elements(By.TAG_NAME, 'img')
                champions = [img.get_attribute("alt") for img in champ_imgs]

                # 중단 조건: Emerald 티어인데 IV가 아님
                if "Emerald" in tier and tier != stop_tier_full:
                    print(f"\n[중단 조건] {tier} 발견 → {stop_tier_full} 이후입니다. 루프 종료 ✅")
                    raise StopIteration

                data = {
                    '순위': rank,
                    '소환사': summoner_tag,
                    '티어': tier,
                    'LP': lp,
                    '레벨': level,
                    '모스트 챔피언': ', '.join(champions),
                    '승률': win_info
                }

                print(f" - [{idx}] {summoner_tag} ({tier}) 수집 ✅")
                data_list.append(data)

            except StopIteration:
                raise
            except Exception as e:
                print(f" - [{idx}] 수집 실패 ❌: {e}")

        page += 1
        time.sleep(2)  # 서버 부하 방지 딜레이

except StopIteration:
    print("[INFO] Emerald IV 티어까지 수집 완료! ")

finally:
    df = pd.DataFrame(data_list)
    today = datetime.now().strftime("%y%m%d")
    filename = f'{today}-opgg_emerald.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n[완료] 총 {len(data_list)}명 저장 완료 👉 CSV 파일: {filename}")
    driver.quit()
df


[페이지 1] 접속 중...
[INFO] 101개의 랭커 항목 수집 중
 - [1] Alpha Smoker
#420 (challenger) 수집 ✅
 - [2] never type
#1998 (challenger) 수집 ✅
 - [3] dmoney
#401K (challenger) 수집 ✅
 - [4] Pentaless
#penta (challenger) 수집 ✅
 - [5] dusklol
#000 (challenger) 수집 ✅
 - [6] Zamudo
#1031 (challenger) 수집 ✅
 - [7] ToastyAlex
#NA1 (challenger) 수집 ✅
 - [8] Mr Dude
#00001 (challenger) 수집 ✅
 - [9] Cupic
#Hwei (challenger) 수집 ✅
 - [10] Will
#NA12 (challenger) 수집 ✅
 - [11] ARMAO
#NA1 (challenger) 수집 ✅
 - [12] Hub1012345
#NA1 (challenger) 수집 ✅
 - [13] blackpanther
#ruin (challenger) 수집 ✅
 - [14] DARKWINGS
#NA3 (challenger) 수집 ✅
 - [15] yukino cat
#cat (challenger) 수집 ✅
 - [16] C9 Loki
#kr3 (challenger) 수집 ✅
 - [17] Sushee
#NA1 (challenger) 수집 ✅
 - [18] Afflictive
#藍月なくる (challenger) 수집 ✅
 - [19] KDKD
#9999 (challenger) 수집 ✅
 - [20] kisno
#NA1 (challenger) 수집 ✅
 - [22] SnowyStepZ
#Saiko (challenger) 수집 ✅
 - [23] perryjg
#quack (challenger) 수집 ✅
 - [24] stupid donut
#YEAH (challenger) 수집 ✅
 - [25] Tkt
#NA33 (challenger) 

,순위,소환사,티어,LP,레벨,모스트 챔피언,승률
0,1,Alpha Smoker\n#420,challenger,"1,992",104,"이즈리얼, 칼리스타, 애쉬",211승\n151패\n58%
1,2,never type\n#1998,challenger,"1,965",370,"리븐, 베인, 제이스",140승\n82패\n63%
2,3,dmoney\n#401K,challenger,"1,929",98,"파이크, 아크샨, 사일러스",241승\n157패\n61%
3,4,Pentaless\n#penta,challenger,"1,920",515,"누누와 윌럼프, 마오카이, 스카너",205승\n143패\n59%
4,5,dusklol\n#000,challenger,"1,840",392,"키아나, 카직스, 릴리아",169승\n117패\n59%
...,...,...,...,...,...,...,...
11795,11796,byungshinnom\n#NA1,diamond 2,68,758,"쓰레쉬, 뽀삐, 렐",31승\n40패\n44%
11796,11797,Ya Boy Drone\n#NA1,diamond 2,68,527,"이즈리얼, 케이틀린, 바이",205승\n206패\n50%
11797,11798,Miskeen Yute\n#000,diamond 2,68,513,"스웨인, 판테온, 마오카이",75승\n75패\n50%
11798,11799,sponge robert\n#bob,diamond 2,68,42,"아크샨, 라이즈, 에코",63승\n37패\n63%
